In [1]:
!git clone https://github.com/Brintsov/YOLOv8-Lightweight-Backbones

Cloning into 'YOLOv8-Lightweight-Backbones'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 104 (delta 19), reused 45 (delta 11), pack-reused 37 (from 1)
Receiving objects: 100% (104/104), 113.30 MiB | 15.55 MiB/s, done.
Resolving deltas: 100% (29/29), done.
Updating files: 100% (20/20), done.


In [6]:
import numpy as np
from keras_cv import visualization
from data.prepate_data import run_data_preparation_pipeline
from models.csp_darknet import create_xs_csp_darknet_yolov8
from models.ghostnet_v2 import create_0_5_ghostnet_yolov8
from models.shufflenet_v2 import create_0_5_shufflenet_yolov8

from keras.callbacks import CSVLogger
from profiling.coco_metric_callback import EvaluateCOCOMetricsCallback
from profiling.profilers import ModelsProfiler, FlopsExtractor, InferenceProfiler, COCOMetricsCalculator
import warnings
warnings.filterwarnings("ignore")

num_classes = 2
input_shape = (640, 640, 3)


In [7]:
gathered_data = run_data_preparation_pipeline()
train_ds, val_ds = gathered_data['train_ds'], gathered_data['val_ds']
cat_mapping_r = gathered_data['cat_mapping_r']
cat_mapping = gathered_data['cat_mapping']

100%|██████████| 1.67G/1.67G [00:17<00:00, 100MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/trainingdatapro/cars-video-object-tracking/versions/3


In [9]:
model_shuffle = create_0_5_shufflenet_yolov8(num_classes=num_classes, input_shape=input_shape)
csp_darknet_yolo = create_xs_csp_darknet_yolov8(num_classes=num_classes)
ghost_model = create_0_5_ghostnet_yolov8(num_classes=num_classes, input_shape=input_shape)

csp_darknet_yolo.load_weights("./YOLOv8-Lightweight-Backbones/best_weights/best_csp_darknet_yolo.keras")
model_shuffle.load_weights("./YOLOv8-Lightweight-Backbones/best_weights/best_shuffle_yolo.keras")
ghost_model.load_weights("./YOLOv8-Lightweight-Backbones/best_weights/best_ghost_yolo.keras")

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 440 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 614 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 540 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
profilers = [
    InferenceProfiler(repeats=250, device='CPU:0'),
    InferenceProfiler(repeats=250, device='GPU:0'),
    FlopsExtractor(),
    COCOMetricsCalculator(val_ds)
]
models = {
    'ShuffleNetV2-YOLOv8': model_shuffle,
    'CSPDarkNet-YOLOv8': csp_darknet_yolo,
    'GhostNetv2-YOLOv8': ghost_model,
}
general_profiles = ModelsProfiler(profilers=profilers)
profile_results = general_profiles.profile(models, input_data)